## Prepare Callbacks

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection'

In [5]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from plantDiseaseDetection.constants import *
from plantDiseaseDetection.utils import create_directories, read_yaml

In [7]:
# Configuration Manager

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) ->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), Path(model_ckpt_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = config.checkpoint_model_filepath
        )

        return prepare_callbacks_config

In [8]:
import time
import tensorflow as tf

2023-11-11 15:36:06.403576: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-11 15:36:06.464326: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 15:36:06.464392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 15:36:06.464448: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 15:36:06.477917: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-11 15:36:06.478818: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
# Component
class PrepareCallback:
    def __init__(self, config=PrepareCallbacksConfig):
        self.config = config

    # Tensorboard Callback
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    # ModelCheckpoint Callback
    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    # EarlyStopping Callback
    @property
    def _create_estp_callback(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            mode="auto"
        )

    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callback,
            self._create_estp_callback
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callback()

except Exception as e:
    raise e

2023-11-11 15:36:08,200 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: config/config.yaml loaded successfully
2023-11-11 15:36:08,202 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: params.yaml loaded successfully
2023-11-11 15:36:08,203 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts
2023-11-11 15:36:08,204 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2023-11-11 15:36:08,205 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts/prepare_callbacks/checkpoint_dir
